In [ ]:
class Value:

    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self._prev = set(_children)
        self._backward = lambda: None # stores function that's why lambda needed
        self._op = _op
        self.grad = 0


    def __add__(self, other):

        other = other if isinstance(other, Value) else Value(other) # handles case where you add a non-Value object
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += 1
            other.grad += 1

        out._backward = _backward # storing the function '_backward()'

        return out

    def __mul__(self, other):

        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad

        out._backward = _backward

        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting integers and floating point numbers"

        out = Value(self.data**other, (self,), f'**{other}')

        def _backward():
            self.grad = other * (self.data**(other - 1))

        out._backward = _backward

        return out

    def __neg__(self):
        return -1 * self.data

    def __sub__(self, other):
        return self + (-other)

    def __truediv__(self, other):
        return self * other**-1

    def __rmul__(self, other):
        return other * self

    def __radd__(self, other):
        return other + self

    def __rsub__(self, other):
        return other + (-self)

    def __rtruediv__(self, other):
        return other * self**-1



        